# 챗봇 문답 데이터 감정 분류 모델 

- 문장을 감정 클래스별로 분류하는 CNN 모델 구현 
- 텍스트 데이터의 임베딩 품질만 괜찮다면 자연어 분류에도 CNN이 좋은 성능을 낼 수 있음
- 컴퓨터는 임베딩된 벡터로 표현 가능한 대상이라면 특징을 추출하도록 CNN 모델을 학습할 수 있음 
- 데이터셋 구조
    - Q(질문)
    - A(답변)
    - label(감정)
        - 0 : 일상다반사
        - 1 : 이별(부정)
        - 2 : 사랑(긍정) 

In [2]:
import numpy as np 
import pandas as pd
import tensorflow as tf 
from tensorflow.keras import preprocessing 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Embedding, Dense, Dropout, Conv1D, 
                                     GlobalMaxPooling1D, concatenate)

##  케라스로 함수형 API로 짤것임

# 데이터 확인

In [4]:
data = pd.read_csv('./data/ChatbotData.csv')
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [5]:
data[data['label'] == 2]

,Q,A,label
8860,짝사랑만큼 고통스러운 건 없겠지.,짝사랑 만큼 감정소모가 큰 건 없을 거예요.,2
8861,1년 넘게 만났는데 지금도 불타올라,정열적인 사랑을 하고 있나봐요.,2
8862,1년 동거 중인데 계속 좋아,서로 깊게 알게되면서 더 좋아졌나봅니다.,2
8863,1년 동거하고 결혼했어,축하합니다!,2
8864,1년 만났는데도 그 사람에 대해 잘 모르겠어,더 만나보세요.,2
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


# 데이터 전처리

In [6]:
features = data['Q'].tolist()

In [7]:
labels = data['label'].tolist()

In [8]:
# 단어 인덱스 시퀀스 벡터 
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]

In [9]:
corpus[1]

['1지망', '학교', '떨어졌어']

In [10]:
tokenizer = preprocessing.text.Tokenizer()

In [11]:
tokenizer.fit_on_texts(corpus)

In [12]:
sequences = tokenizer.texts_to_sequences(corpus)

In [13]:
sequences[1]

[4648, 343, 448]

In [21]:
word_index= tokenizer.word_index

In [22]:
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen= MAX_SEQ_LEN,
                                                   padding = 'post')

In [23]:
padded_seqs[0]

array([4646, 4647,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0])

In [24]:
# 학습용 검증용 테스트용 데이터셋 생성
# 학습 : 검증 : 테스트 = 7 : 2 : 1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

In [25]:
ds.batch

<bound method DatasetV2.batch of <_ShuffleDataset element_spec=(TensorSpec(shape=(15,), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>>

In [26]:
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

In [27]:
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 모델 구성

In [28]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1 # 전체 단어 수

In [33]:
# CNN 모델 정의
input_layer = Input(shape = (MAX_SEQ_LEN,))
embedding_layer= Embedding(VOCAB_SIZE, EMB_SIZE, input_shape = (MAX_SEQ_LEN,))(input_layer)
dropout_emb = Dropout(rate = dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters= 128,
    kernel_size= 3,
    padding= 'valid',
    activation = tf.nn.relu
)(dropout_emb)
pool1 = GlobalMaxPooling1D()(conv1)

conv2 = Conv1D(
    filters = 128, # 채널들
    kernel_size= 4,
    padding= 'valid',
    activation = tf.nn.relu
)(dropout_emb)
pool2 = GlobalMaxPooling1D()(conv2)

conv3 = Conv1D(
    filters = 128, 
    kernel_size= 5,
    padding= 'valid',
    activation = tf.nn.relu
)(dropout_emb)
pool3 = GlobalMaxPooling1D()(conv3)

# 3, 4, 5 - gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation = tf.nn.relu)(concat)
dropout_hidden = Dropout(rate = dropout_prob)(hidden)
logits = Dense(3, name = 'logits')(dropout_hidden)
predictions = Dense(3, activation = tf.nn.softmax)(logits)

- Maxpool 하나하나 풀링크기에서 최대값을 뽑아내고
- GlobalMaxPooling1D : 전체의 최대값을 뽑아냄, CNN에서 많이 사용됨 
- 시퀀스  → Embedding -> Dense -> Dropout -> Dense -> Dense

In [34]:
# 모델 생성
model = Model(inputs = input_layer, outputs = predictions)

In [35]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)    │ (None, 15)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_3 (Embedding)       │ (None, 15, 128)           │       1,715,072 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 15, 128)           │               0 │ embedding_3[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 13, 128)           │          49,280 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, 12, 128)           │          65,664 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_2 (Conv1D)             │ (None, 11, 128)           │          82,048 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d          │ (None, 128)               │               0 │ conv1d[0][0]               │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d_1        │ (None, 128)               │               0 │ conv1d_1[0][0]             │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d_2        │ (None, 128)               │               0 │ conv1d_2[0][0]             │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 384)               │               0 │ global_max_pooling1d[0][0… │
│                               │                           │                 │ global_max_pooling1d_1[0]… │
│                               │                           │                 │ global_max_pooling1d_2[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │          49,280 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 3)                 │             387 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 3)                 │              12 │ logits[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,961,743 (7.48 MB)

 Trainable params: 1,961,743 (7.48 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
model.compile(optimizer= 'adam', loss='sparse_categorical_crossentropy', metrics= ['accuracy'])

# 모델 학습

In [37]:
model.fit(train_ds, validation_data= val_ds, epochs= EPOCH)

Epoch 1/5
414/414 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.5167 - loss: 0.9795 - val_accuracy: 0.6087 - val_loss: 0.7524
Epoch 2/5
414/414 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.6741 - loss: 0.6984 - val_accuracy: 0.8921 - val_loss: 0.3735
Epoch 3/5
414/414 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.8686 - loss: 0.3779 - val_accuracy: 0.9492 - val_loss: 0.1814
Epoch 4/5
414/414 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.9285 - loss: 0.2261 - val_accuracy: 0.9700 - val_loss: 0.1004
Epoch 5/5
414/414 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.9578 - loss: 0.1322 - val_accuracy: 0.9788 - val_loss: 0.0719


# 모델평가

In [39]:
model.evaluate(test_ds)

60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9744 - loss: 0.0660


[0.07325051724910736, 0.9746192693710327]

In [38]:
model.save('./model/cnn_model.h5')

In [40]:
np.random.randint(0, len(corpus))

10042

In [41]:
corpus[10882]

['유학', '준비하는', '여자친구']

In [42]:
labels[10882]

2

In [43]:
model.predict(padded_seqs[[10882]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step


array([[1.0193168e-04, 1.3893195e-03, 9.9850875e-01]], dtype=float32)